# CHAPTER 6 실습: 상용 배포 준비

이 노트북은 『MLOps 도입 가이드』 6장의 내용을 기반으로, 상용 배포 전 자동화 및 패키징을 실습합니다.

실습 항목:
- 모델 학습 및 저장
- MLflow 모델 로깅
- Dockerfile 생성 가이드 출력

## 1. 모델 학습 및 저장

In [ ]:
import pandas as pd
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
import joblib

# 데이터 로딩 및 전처리
df = sns.load_dataset('titanic').dropna(subset=['age','embarked'])
df['sex'] = df['sex'].map({'male':0,'female':1})
df['embarked'] = df['embarked'].astype('category').cat.codes
features = ['pclass','sex','age','sibsp','parch','fare','embarked']
X = df[features]
y = df['survived']

# 모델 학습
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X, y)

# 모델 저장
joblib.dump(model, 'model_rf.pkl')
print('✅ 모델 저장 완료: model_rf.pkl')

## 2. MLflow 모델 로깅

In [ ]:
import mlflow
import mlflow.sklearn

mlflow.set_experiment('deployment_test')
with mlflow.start_run():
    mlflow.log_param('n_estimators', 100)
    mlflow.log_metric('accuracy', model.score(X, y))
    mlflow.sklearn.log_model(model, 'model')
print('✅ MLflow 모델 로깅 완료')

## 3. Dockerfile 예시

In [ ]:
dockerfile = '''
# Dockerfile for model serving
FROM python:3.11-slim
WORKDIR /app

COPY model_rf.pkl ./
COPY requirements.txt ./
RUN pip install -r requirements.txt

CMD ['python', 'serve.py']
'''
print(dockerfile)

🔧 `requirements.txt` 및 `serve.py` 파일은 별도로 작성하여 컨테이너에서 모델 서빙 API를 구현하세요.

- 예: Flask, FastAPI를 이용해 `model_rf.pkl` 로드 후 `/predict` 엔드포인트 생성
- `requirements.txt`에는 `scikit-learn`, `mlflow`, `joblib`, `flask` 등 포함 가능

이 실습은 배포 전 모델 패키징 및 아티팩트 관리의 기초를 보여줍니다.